Ask for the definition of a medical term, and get a response that is reconciled between multiple web sources.  For an MVP, the web app will allow the user to enter 'heart attack' and get a machine generated reconciled definition from Wikipedia, WebMD, and MayoClinic.  Future iterations would include adding the ability to search more medical terms, using more sources with which to reconcile the requested definition, as well as including symptoms, causes, treatments, and risk factors.  Additional future iterations will include comparing sources with conflicting results with the established "training set" of sources.  These conflicting sources can be documented for additional scrutinization.

In [1]:
import requests
from bs4 import BeautifulSoup
import re
import nltk
from nltk.tokenize import sent_tokenize
from sklearn.feature_extraction.text import CountVectorizer
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
import matplotlib.pyplot as plt

In [2]:
import numpy as np
#import pandas as pd
#import nltk
#import re
import os
import codecs
#from sklearn import feature_extraction
import mpld3
import seaborn as sb 

In [3]:
#word2vec packages
import gensim
from gensim import corpora, models, similarities
import logging
import os

/Users/markregalla/anaconda/lib/python2.7/site-packages/numpy/lib/utils.py:95: DeprecationWarning: `scipy.sparse.sparsetools` is deprecated!
scipy.sparse.sparsetools is a private module for scipy.sparse, and should not be used.
  warnings.warn(depdoc, DeprecationWarning)
/Users/markregalla/anaconda/lib/python2.7/site-packages/scipy/lib/_util.py:67: DeprecationWarning: Module scipy.linalg.blas.fblas is deprecated, use scipy.linalg.blas instead
  DeprecationWarning)
/Users/markregalla/anaconda/lib/python2.7/site-packages/scipy/lib/_util.py:67: DeprecationWarning: Module scipy.linalg.blas.fblas is deprecated, use scipy.linalg.blas instead
  DeprecationWarning)
/Users/markregalla/anaconda/lib/python2.7/site-packages/scipy/lib/_util.py:67: DeprecationWarning: Module scipy.linalg.blas.fblas is deprecated, use scipy.linalg.blas instead
  DeprecationWarning)
/Users/markregalla/anaconda/lib/python2.7/site-packages/scipy/lib/_util.py:67: DeprecationWarning: Module scipy.linalg.blas.fblas is depr

In [4]:
import numpy as np
from sklearn.metrics import jaccard_similarity_score
from scipy import spatial

Keywords to use when searching for the definition of a medical term:

'term' is

'term' occurs

###Wikipedia

In [5]:
response = requests.get('https://en.wikipedia.org/wiki/Myocardial_infarction')
page = response.text
soup = BeautifulSoup(page, 'html.parser')

In [6]:
paragraphs = soup.findAll('p')
wikiparagraphs = []
k = 0
theindex = 0
foundone = False
for i in paragraphs:
    wikiparagraphs.append(i.text)
    if 'heart attack' in paragraphs[k].text:
        if foundone == False:
            theindex = k
            foundone = True
    k += 1
wikidef = paragraphs[theindex].text
wikisentences = sent_tokenize(wikidef)

def cleanText(corpus):
    punctuation = """.,?!:;(){}[]"""
    corpus = [z.lower().replace('\n','') for z in corpus]
    #corpus = [z.replace('<br />', ' ') for z in corpus]

    #treat punctuation as individual words
    for c in punctuation:
        corpus = [z.replace(c, ' %s '%c) for z in corpus]
    corpus = [z.split() for z in corpus]
    return corpus

wikiparagraphs = cleanText(wikiparagraphs)
wikiparagraphs

[[u'myocardial',
  u'infarction',
  u'(',
  u'mi',
  u')',
  u'or',
  u'acute',
  u'myocardial',
  u'infarction',
  u'(',
  u'ami',
  u')',
  u',',
  u'commonly',
  u'known',
  u'as',
  u'a',
  u'heart',
  u'attack',
  u',',
  u'occurs',
  u'when',
  u'blood',
  u'flow',
  u'stops',
  u'to',
  u'a',
  u'part',
  u'of',
  u'the',
  u'heart',
  u'causing',
  u'damage',
  u'to',
  u'the',
  u'heart',
  u'muscle',
  u'.',
  u'the',
  u'most',
  u'common',
  u'symptom',
  u'is',
  u'chest',
  u'pain',
  u'or',
  u'discomfort',
  u'which',
  u'may',
  u'travel',
  u'into',
  u'the',
  u'shoulder',
  u',',
  u'arm',
  u',',
  u'back',
  u',',
  u'neck',
  u',',
  u'or',
  u'jaw',
  u'.',
  u'often',
  u'it',
  u'is',
  u'in',
  u'the',
  u'center',
  u'or',
  u'left',
  u'side',
  u'of',
  u'the',
  u'chest',
  u'and',
  u'lasts',
  u'for',
  u'more',
  u'than',
  u'a',
  u'few',
  u'minutes',
  u'.',
  u'the',
  u'discomfort',
  u'may',
  u'occasionally',
  u'feel',
  u'like',
  u'heartburn'

###WebMD

In [7]:
response2 = requests.get('http://www.webmd.com/heart-disease/guide/heart-disease-heart-attacks')
page2 = response2.text
soup2 = BeautifulSoup(page2, 'html.parser')

In [9]:
paragraphs2 = soup2.findAll('p')
k = 0
theindex2 = 0
foundone2 = False
for i in paragraphs2:
    if 'heart attack' in paragraphs2[k].text:
        if foundone2 == False:
            theindex2 = k
            foundone2 = True
    k += 1
webmddef = paragraphs2[theindex2].text
webmdsentences = sent_tokenize(webmddef)
webmddef
TaggedDocument = gensim.models.doc2vec.TaggedDocument
webmdlist = []
p = 0
for i in paragraphs:
    #webmdlist.append((p, i.text))
    webmdlist.append(TaggedDocument(paragraphs[0].text, p))
    p += 1

###MayoClinic

In [10]:
response3 = requests.get('http://www.mayoclinic.org/diseases-conditions/heart-attack/basics/definition/con-20019520')
page3 = response3.text
soup3 = BeautifulSoup(page3, 'html.parser')

In [11]:
paragraphs3 = soup3.findAll('p')
k = 0
theindex3 = 0
foundone3 = False
for i in paragraphs3:
    if 'heart attack' in paragraphs3[k].text:
        if foundone3 == False:
            theindex3 = k
            foundone3 = True
    k += 1
mayodef = paragraphs3[theindex3].text
mayosentences = sent_tokenize(mayodef)
mayodef
#TaggedDocument = gensim.models.doc2vec.TaggedDocument
mayolist = []
p = 0
for i in paragraphs:
    #mayolist.append((p, i.text))
    mayolist.append(TaggedDocument(paragraphs[0].text, p))
    p += 1

#Compare all three definitions
https://www.youtube.com/watch?v=FfLo5OHBwTo

In [33]:
allthree = [wikidef, webmddef, mayodef]
vectorizer = CountVectorizer()
allthree = sorted(allthree)
X = vectorizer.fit_transform(allthree)
#print X
X_Array = X.toarray()
#X_Array
#analyze = vectorizer.build_analyzer()
#analyze(wikidef) == wikidef

In [34]:
#pd.set_option('display.max_rows', 5)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
comparedf = pd.DataFrame(X_Array, columns=vectorizer.get_feature_names())
comparedf = comparedf.transpose()
comparedf.columns = ['Wiki', 'WebMD', 'MayoClinic']
comparedf

,Wiki,WebMD,MayoClinic
30,0,0,1
75,0,0,1
about,0,0,2
acute,0,0,1
americans,0,1,0
ami,0,0,1
among,0,0,1
an,0,0,3
and,1,1,1
arm,0,0,1


In [13]:
#comparedf[comparedf['Wiki'] & comparedf['WebMD'] & comparedf['MayoClinic'] != 0]
comparedf[comparedf['Wiki'] & comparedf['WebMD'] != 0]


,Wiki,WebMD,MayoClinic
and,1,1,1
attack,1,1,1
damage,1,1,1
heart,4,4,4
is,1,1,2
of,3,2,5
or,1,1,7
the,6,2,7
to,1,3,3


###Jaccard Similarity

In [11]:
#Jaccard Similarity by comparing whole paragraph
#jaccard_similarity_score(wikidef, webmddef, normalize=False)
wikiset = set(wikidef)
webmdset = set(webmddef)
mayoset = set(mayodef)

theintersect = wikiset.intersection(webmdset)
theintersect = theintersect.intersection(mayodef)
theunion = wikiset.union(webmdset)
theunion = theunion.union(mayodef)

theintersect_len = len(list(theintersect))
theunion_len = len(list(theunion))

jaccard = float(theintersect_len) / theunion_len
if jaccard > 0.5:
    print jaccard
    astring = 'Definitions Match.\n'
    astring = astring + 'Wikipedia Definition (as confirmed by WebMD and MayoClinic)\n'
    astring = astring + wikidef
    print astring
print type(jaccard)

0.553191489362
Definitions Match.
Wikipedia Definition (as confirmed by WebMD and MayoClinic)
Myocardial infarction (MI) or acute myocardial infarction (AMI), commonly known as a heart attack, occurs when blood flow stops to a part of the heart causing damage to the heart muscle. The most common symptom is chest pain or discomfort which may travel into the shoulder, arm, back, neck, or jaw. Often it is in the center or left side of the chest and lasts for more than a few minutes. The discomfort may occasionally feel like heartburn. Other symptoms may include shortness of breath, nausea, feeling faint, a cold sweat, or feeling tired.[1] About 30% of people have atypical symptoms,[2] with women more likely than men to present atypically.[3] Among those over 75 years old, about 5% have had an MI with little or no history of symptoms.[4] An MI may cause heart failure, an irregular heartbeat, or cardiac arrest.[5][6]
<type 'float'>


In [12]:
#Jaccard similarity find best matching sentences

theintersect = wikiset.intersection(webmdset)
theintersect = theintersect.intersection(mayodef)
theunion = wikiset.union(webmdset)
theunion = theunion.union(mayodef)

{u' ',
 u'"',
 u'%',
 u'(',
 u')',
 u',',
 u'-',
 u'.',
 u'0',
 u'1',
 u'2',
 u'3',
 u'4',
 u'5',
 u'6',
 u'7',
 u'A',
 u'I',
 u'M',
 u'O',
 u'T',
 u'[',
 u']',
 u'a',
 u'b',
 u'c',
 u'd',
 u'e',
 u'f',
 u'g',
 u'h',
 u'i',
 u'j',
 u'k',
 u'l',
 u'm',
 u'n',
 u'o',
 u'p',
 u'q',
 u'r',
 u's',
 u't',
 u'u',
 u'v',
 u'w',
 u'y'}

In [24]:
from nltk.corpus import wordnet as wn
from nltk.tokenize import word_tokenize

def max_wupa(context_sentence, ambiguous_word):
  """ 
  WSD by Maximizing Wu-Palmer Similarity.

  Perform WSD by maximizing the sum of maximum Wu-Palmer score between possible 
  synsets of all words in the context sentence and the possible synsets of the 
  ambiguous words (see http://goo.gl/XMq2BI):
  {argmax}_{synset(a)}(\sum_{i}^{n}{{max}_{synset(i)}(Wu-Palmer(i,a))}

  Wu-Palmer (1994) similarity is based on path length; the similarity between 
  two synsets accounts for the number of nodes along the shortest path between 
  them. (see http://acl.ldc.upenn.edu/P/P94/P94-1019.pdf)
  """

  result = {}
  for i in wn.synsets(ambiguous_word):
    result[i] = sum(max([i.wup_similarity(k) for k in wn.synsets(j)]+[0]) \
                    for j in word_tokenize(context_sentence))
  result = sorted([(v,k) for k,v in result.items()],reverse=True)
  return result

bank_sents = ['I went to the bank to deposit my money',
'The river bank was full of dead fishes']
ans = max_wupa(bank_sents[0], 'bank')
print ans
print ans[0][1].definition

[(2.833333333333333, Synset('deposit.v.02')), (2.571428571428571, Synset('bank.n.06')), (2.3523809523809525, Synset('bank.n.09')), (2.2675324675324675, Synset('bank.v.07')), (2.2675324675324675, Synset('bank.v.05')), (2.160839160839161, Synset('bank.n.04')), (2.1, Synset('bank.n.05')), (2.0857142857142854, Synset('savings_bank.n.02')), (1.994871794871795, Synset('depository_financial_institution.n.01')), (1.9487179487179485, Synset('bank.n.07')), (1.9487179487179485, Synset('bank.n.01')), (1.9474969474969472, Synset('bank.v.04')), (1.9474969474969472, Synset('bank.v.03')), (1.9474969474969472, Synset('bank.v.02')), (1.9070048309178742, Synset('bank.n.10')), (1.879120879120879, Synset('bank.n.03')), (1.8428571428571427, Synset('trust.v.01')), (1.8428571428571427, Synset('bank.v.01'))]
<bound method Synset.definition of Synset('deposit.v.02')>


###Cosine Similarity

In [42]:
#spatial.distance.cosine(wikidef, webmddef)
import math
import numpy as np
def get_cosine(text1, text2):
  vec1 = text1
  vec2 = text2
  intersection = set(vec1) & set(vec2)
  numerator = sum([vec1[x] * vec2[x] for x in intersection])
  sum1 = sum([vec1[x]**2 for x in vec1])
  sum2 = sum([vec2[x]**2 for x in vec2])
  denominator = math.sqrt(sum1) * math.sqrt(sum2)
  if not denominator:
     return 0.0
  else:
     return round(float(numerator) / denominator, 3)
#get_cosine(wikimodel['heart'], webmdmodel['heart'])
npa1 = np.array('poo', 'fart')
npa2 = np.array('poo', 'fart')
get_cosine(npa1, npa2)

TypeError: data type "fart" not understood

###Word to Vector

In [12]:
stoplist = set('for a of the and to in an or'.split())

#Wikipedia
wikitext = [[word for word in wikidef.lower().split() if word not in stoplist]]

#WebMD
webmdtexts = [[word for word in webmddef.lower().split() if word not in stoplist]]

#MayoClinic
mayotexts = [[word for word in mayodef.lower().split() if word not in stoplist]]

In [13]:
wikiset = set(wikidef)
webmdset = set(webmddef)
mayoset = set(mayodef)

In [16]:
%matplotlib inline
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

wikiwordmodel = gensim.models.Word2Vec(wikitext, size=100, window=5, min_count=1, workers=4, sg=1)
webmdwordmodel = gensim.models.Word2Vec(webmdtexts, size=100, window=5, min_count=1, workers=4, sg=1)
mayowordmodel = gensim.models.Word2Vec(mayotexts, size=100, window=5, min_count=1, workers=4, sg=1)

#dictionary = corpora.Dictionary(texts)
#corpus = [dictionary.doc2bow(text) for text in texts]
#print model['blood']
#plt.hist(model['blood'], bins=20)
#model.most_similar(positive=['stops', 'heart'], topn=5)

#print dictionary.token2id

#print corpus
#print model.vocab['over']
#print wikimodel['heart']
#print webmdmodel['heart']
#print type(wikimodel['heart'])
#print spatial.distance.cosine(wikimodel['myocardial'], webmdmodel['heart'])
print wikiwordmodel

#theintersect = wikiset.intersection(webmdset)
#theintersect = theintersect.intersection(mayodef)
'''
vectorlist = []
i = 0
while i < len(texts[0]):
    k = 0
    while k < len(texts2[0])
        if texts[0][i] == texts2[0][k]:
            vectorlist.append(wikimodel[texts[0][i]])
model['heart']
model.most_similar('heart')
'''

Word2Vec(vocab=91, size=100, alpha=0.025)


"\nvectorlist = []\ni = 0\nwhile i < len(texts[0]):\n    k = 0\n    while k < len(texts2[0])\n        if texts[0][i] == texts2[0][k]:\n            vectorlist.append(wikimodel[texts[0][i]])\nmodel['heart']\nmodel.most_similar('heart')\n"

In [ ]:
'''
class MySentences(object):
     def __init__(self, dirname):
        self.dirname = dirname
 
     def __iter__(self):
         for fname in os.listdir(self.dirname):
                for line in open(os.path.join(self.dirname, fname)):
                    yield line.split()
'''
#sentences = MySentences('/Users/markregalla/nltk_data/corpora/gutenberg') # a memory-friendly iterator
model = gensim.models.Word2Vec(texts, min_count=3,workers=5)
#model.most_similar(positive=['attack'], negative=['heart'], topn=5)
new_doc = "myocardial"
new_vec = dictionary.doc2bow(new_doc.lower().split())
print new_vec # the word "interaction" does not appear in the dictionary and is ignored

###Document to Vector

In [32]:
#listwikidef = [wikidef]
TaggedDocument = gensim.models.doc2vec.TaggedDocument
wikidocmodel = gensim.models.Doc2Vec(size=100, window=8, min_count=5, workers=4)

#make a list containing tuples of index and paragraph
#labels are needed for doc2vec
def labelizeReviews(reviews, label_type):
    labelized = []
    for i,v in enumerate(reviews):
        label = '%s_%s'%(label_type,i)
        labelized.append(TaggedDocument(v, [label]))
    return labelized
#listwikidef = labelizeReviews(wikiparagraphs, 'TRAIN')
listwikidef = labelizeReviews(wikiparagraphs, 'TRAIN')
wikidocmodel.build_vocab(listwikidef)
#.similarity('heart', 'blood')

#now what can I do with this model?
'''
is this model considered the standard of comparison?
do I take the other sources and run its text through this model as a test set?
What is produced when this model runs? Another vector?
Is another vector produced such that I can see a comparison score relative to the training set?
Is that comparison score done using cosine similarity?  other similarity metrics?
'''
print wikidocmodel.most_similar("myocardial")

[(u'fibrillation', 0.266294002532959), (u'with', 0.2628459930419922), (u'used', 0.24140343070030212), (u'flow', 0.23066675662994385), (u'arteries', 0.22322329878807068), (u'increased', 0.2131156623363495), (u'countries', 0.20402422547340393), (u'muscle', 0.2013978362083435), (u'causes', 0.182762011885643), (u'15', 0.17207512259483337)]


###LSI

In [55]:
doc_bow = [(0, 1), (1, 1)]
lsi = models.LsiModel(corpus, id2word=dictionary, num_topics=2)
tfidf = models.TfidfModel(corpus)
corpus_tfidf = tfidf[corpus]
corpus_lsi = lsi[corpus_tfidf]
for doc in corpus_lsi:
    print doc

[]


In [35]:
lsi.print_topics()

[(0,
  u'0.319*"may" + 0.319*"heart" + 0.160*"infarction" + 0.160*"than" + 0.160*"more" + 0.160*"have" + 0.160*"feeling" + 0.160*"myocardial" + 0.160*"with" + 0.160*"chest"')]

In [54]:
vec_bow = dictionary.doc2bow(texts2)
vec_lsi = lsi[vec_bow] # convert the query to LSI space
print(vec_lsi)
[(0, -0.461821), (1, 0.070028)]

TypeError: coercing to Unicode: need string or buffer, list found

###FuzzyWuzzy

In [22]:
print fuzz.token_set_ratio(wikisentences[0], mayosentences[0])

58


In [16]:
wikisentences[0]

u'Myocardial infarction (MI) or acute myocardial infarction (AMI), commonly known as a heart attack, occurs when blood flow stops to a part of the heart causing damage to the heart muscle.'

In [17]:
mayosentences[0]

u'A heart attack occurs when the flow of blood to the heart is blocked, most often by a build-up of fat, cholesterol and other substances, which form a plaque in the arteries that feed the heart (coronary arteries).'